In [9]:
import pandas as pd

# Load the datasets
landed_costs_path = 'C:/Users/olcay/Desktop/TASKS/TrakRacer/AUS Shipping Comparison/Take a look later on/Landed Costs AUS.xlsx'
actual_product_price_path = 'C:/Users/olcay/Desktop/TASKS/TrakRacer/AUS Shipping Comparison/Take a look later on/Actual Product Price.xlsx'
shipment_data_path = 'C:/Users/olcay/Desktop/TASKS/TrakRacer/AUS Shipping Comparison/Updated_Data_with_Logistic_Regression_Predictions.csv'  # Add the correct path for the shipment data CSV

# Read the data from the Excel files
landed_costs_data = pd.read_excel(landed_costs_path, sheet_name='All Product Cost')
actual_product_price_data = pd.read_excel(actual_product_price_path, sheet_name='Products')
shipment_data = pd.read_csv(shipment_data_path)

# Extract the necessary columns
landed_costs = landed_costs_data[['SKU', 'Landed Cost to AU']]
product_prices = actual_product_price_data[['Variant SKU', 'Variant Weight', 'Variant Price']]

# Merge the datasets on SKU
merged_data = pd.merge(landed_costs, product_prices, left_on='SKU', right_on='Variant SKU', how='inner')

# Calculate the adjusted final price using the weighted average shipping cost per kg
weighted_avg_shipping_cost_per_kg = 1.63  # The previously calculated average shipping cost per kg
merged_data['Adjusted Price'] = merged_data['Variant Price'] + (merged_data['Variant Weight'] * weighted_avg_shipping_cost_per_kg)

# Calculate initial and adjusted gross margins
merged_data['Initial Gross Margin'] = merged_data['Variant Price'] - merged_data['Landed Cost to AU']
merged_data['Adjusted Gross Margin'] = merged_data['Adjusted Price'] - merged_data['Landed Cost to AU']

# Process the shipment data to calculate total shipping costs per SKU
shipment_data['SKUs'] = shipment_data['Product'].str.split(',')
shipment_data_exploded = shipment_data.explode('SKUs')
shipment_data_exploded['SKUs'] = shipment_data_exploded['SKUs'].str.strip()

# Merge shipment data with SKU to get the relevant shipping costs
shipping_costs_merged = pd.merge(shipment_data_exploded, landed_costs, left_on='SKUs', right_on='SKU', how='inner')
shipping_costs_per_sku = shipping_costs_merged.groupby('SKU').agg(total_shipping_cost=('Total_Charge', 'sum')).reset_index()

# Merge shipping costs back into the main dataset

final_merged_data = pd.merge(merged_data, shipping_costs_per_sku, left_on='SKU', right_on='SKU', how='left')

# Display the first few rows of the final merged data
print(final_merged_data.head())

# Save the final merged data to a new Excel file
final_merged_data.to_excel('Final_Merged_Data_with_Margins_and_Shipping_Costs.xlsx', index=False)

          SKU  Landed Cost to AU Variant SKU  Variant Weight  Variant Price  \
0  941-000152         386.630000  941-000152            7.15          594.0   
1  943-000125          41.420000  943-000125            0.36           39.0   
2  945-000024         197.230000  945-000024            3.52          287.0   
3  960-001090         132.530000  960-001090            0.34          157.0   
4    CSX3BLK4        1060.736343    CSX3BLK4            0.01         2449.0   

   Adjusted Price  Initial Gross Margin  Adjusted Gross Margin  \
0        605.6545            207.370000             219.024500   
1         39.5868             -2.420000              -1.833200   
2        292.7376             89.770000              95.507600   
3        157.5542             24.470000              25.024200   
4       2449.0163           1388.263657            1388.279957   

   total_shipping_cost  
0                  NaN  
1                  NaN  
2                  NaN  
3                  NaN  
4  

In [ ]:
final_merged_data['Adjusted Gross Margin'].sum()

Missing Values

In [6]:
# Check the total number of unique SKUs in the "Landed Costs AUS" dataset
total_skus_landed_costs = landed_costs['SKU'].nunique()

# Check the total number of unique SKUs in the merged dataset
total_skus_merged = merged_data['SKU'].nunique()

# Identify missing SKUs
missing_skus = set(landed_costs['SKU']) - set(merged_data['SKU'])

# Display the results
print(f"Total unique SKUs in 'Landed Costs AUS' dataset: {total_skus_landed_costs}")
print(f"Total unique SKUs in the merged dataset: {total_skus_merged}")
print(f"Number of missing SKUs: {len(missing_skus)}")

# If there are missing SKUs, display them
if missing_skus:
    print("Missing SKUs:") 
    print(missing_skus)

Total unique SKUs in 'Landed Costs AUS' dataset: 720
Total unique SKUs in the merged dataset: 329
Number of missing SKUs: 391
Missing SKUs:
{'TR80-TMARM2-BLK', '008011RNR', 'TR160-AINV3-CA', 'TRX-ALP-PART1', 'TR80L-WM-CA', 'SP-ENDCAP-4080', '008013RNR', 'SP-TR80-50', 'TR160S-L-4PBAL', 'TR80-NWMA-WM4', 'TR160-PEDALUP3', 'S01H4R', 'TR80-LITE-PART2', 'TR80-RUBS-BLUE', 'TR-MSW', 'VNM-S01PCB', 'MSI-PC2', 'SCAP-SET-2', 'TR120-AINV3USM2', 'TR120-WMPBAL', 'TRX-A-TK', 'TR160S-4PBAL', 'TR80-160B-BLACK', 'FS3-02', 'VNM-HBRL-CFP', 'MS-B33', 'TR160-AINV3', 'TR-SPMT-FS3-01', 'TR160S-L-AINV3', 'TR160-WMPBAL-DB', 'TR160S-DDPBAL', 'SL-BMW-TR', 'TR120-4PBNPSMEX', 'TRX-BLK23', 'DBOX-3250i', 'TK-SING-UPS', 'TR160-4PBNP-CA', 'TR80-160B-RED-GLOSS', 'RS6-FLT-NS', 'VNM-S01LBS', 'DBOX-4-BASE', 'SP-TRMAT-CAB', 'TR120-WMINV3SMEX', 'TR8-06-G-PART1', 'TR80-MM2-BLK', 'SC2ULT-TR-DD', 'TR120-APBALUSM2', 'TR120-WMINV3', 'TR160S-DDINV3', 'TR160S-BLK-PART1', 'FCORECLUTCHES', 'TR80-HPH', 'MS-FM-SML', '822-0038', 'SP-TR-T

# Price Adjustment Factor : Weighted Average Comparison Method

In [1]:
import pandas as pd

# Load the dataset
shipment_data_path = 'C:/Users/olcay/Desktop/TASKS/TrakRacer/AUS Shipping Comparison/Updated_Data_with_Logistic_Regression_Predictions.csv'
data = pd.read_csv(shipment_data_path)

# Define the states that are not free shipping
non_free_shipping_states = ['Western Australia', 'Northern Territory', 'Tasmania', 'New Zealand']

# Filter the dataset to exclude non-free shipping states
free_shipping_data = data[~data['State'].isin(non_free_shipping_states)]

# Calculate the shipping cost per kg for free shipping states
free_shipping_data['Shipping_Cost_per_Kg'] = free_shipping_data['Total_Charge'] / free_shipping_data['Weight']

# Group by state to calculate the total weight and total shipping cost
free_shipping_state_analysis = free_shipping_data.groupby('State').agg(
    total_weight=('Weight', 'sum'),
    total_shipping_cost=('Total_Charge', 'sum')
).reset_index()

# Calculate the weighted average shipping cost per kg for free shipping states
total_weight_free_shipping = free_shipping_state_analysis['total_weight'].sum()
total_shipping_cost_free_shipping = free_shipping_state_analysis['total_shipping_cost'].sum()
weighted_avg_shipping_cost_per_kg_free_shipping = total_shipping_cost_free_shipping / total_weight_free_shipping

# Calculate the shipping cost per kg for all states
data['Shipping_Cost_per_Kg'] = data['Total_Charge'] / data['Weight']

# Group by state to calculate the total weight and total shipping cost
all_state_analysis = data.groupby('State').agg(
    total_weight=('Weight', 'sum'),
    total_shipping_cost=('Total_Charge', 'sum')
).reset_index()

# Calculate the weighted average shipping cost per kg for all states
total_weight_all_states = all_state_analysis['total_weight'].sum()
total_shipping_cost_all_states = all_state_analysis['total_shipping_cost'].sum()
weighted_avg_shipping_cost_per_kg_all_states = total_shipping_cost_all_states / total_weight_all_states

# Display results
print(f"Weighted Average Shipping Cost per Kg (Free Shipping States Only): {weighted_avg_shipping_cost_per_kg_free_shipping:.2f}")
print(f"Weighted Average Shipping Cost per Kg (All States): {weighted_avg_shipping_cost_per_kg_all_states:.2f}")


Weighted Average Shipping Cost per Kg (Free Shipping States Only): 1.63
Weighted Average Shipping Cost per Kg (All States): 1.86


C:\Users\olcay\AppData\Local\Temp\ipykernel_17940\817370606.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  free_shipping_data['Shipping_Cost_per_Kg'] = free_shipping_data['Total_Charge'] / free_shipping_data['Weight']


Validity of the 1.63 

# Average Initial Gross Margin

In [8]:
import pandas as pd

# Load the data from the provided Excel file
file_path = 'C:/Users/olcay/Desktop/TASKS/TrakRacer/AUS Shipping Comparison//Final_Merged_Data_with_Margins_and_Shipping_Costs.xlsx'
data = pd.read_excel(file_path)

# Calculate the average initial and adjusted gross margins
average_initial_gross_margin = data['Initial Gross Margin'].mean()
average_adjusted_gross_margin = data['Adjusted Gross Margin'].mean()

# Calculate the average increase in gross margin
average_gross_margin_increase = average_adjusted_gross_margin - average_initial_gross_margin

# Calculate the average shipping cost per order
average_shipping_cost_per_order = data['total_shipping_cost'].mean()

# Display results
print(f"Average Initial Gross Margin: {average_initial_gross_margin:.2f}")
print(f"Average Adjusted Gross Margin: {average_adjusted_gross_margin:.2f}")
print(f"Average Increase in Gross Margin: {average_gross_margin_increase:.2f}")
print(f"Average Shipping Cost per Order: {average_shipping_cost_per_order:.2f}")

# Save the results to a new Excel file
summary_data = {
    'Metric': ['Average Initial Gross Margin', 'Average Adjusted Gross Margin', 'Average Increase in Gross Margin', 'Average Shipping Cost per Order'],
    'Value': [average_initial_gross_margin, average_adjusted_gross_margin, average_gross_margin_increase, average_shipping_cost_per_order]
}
summary_df = pd.DataFrame(summary_data)
summary_df.to_excel('Gross_Margin_Comparison_Summary.xlsx', index=False)

Average Initial Gross Margin: 465.00
Average Adjusted Gross Margin: 590.87
Average Increase in Gross Margin: 125.88
Average Shipping Cost per Order: 1039.05


In [17]:
import pandas as pd

# Load the datasets
landed_costs_path = 'C:/Users/olcay/Desktop/TASKS/TrakRacer/AUS Shipping Comparison/Take a look later on/Landed Costs AUS.xlsx'
actual_product_price_path = 'C:/Users/olcay/Desktop/TASKS/TrakRacer/AUS Shipping Comparison/Take a look later on/Actual Product Price.xlsx'
shipment_data_path = 'C:/Users/olcay/Desktop/TASKS/TrakRacer/AUS Shipping Comparison/Updated_Data_with_Logistic_Regression_Predictions.csv'

# Read the data from the Excel files
landed_costs_data = pd.read_excel(landed_costs_path, sheet_name='All Product Cost')
actual_product_price_data = pd.read_excel(actual_product_price_path, sheet_name='Products')
shipment_data = pd.read_csv(shipment_data_path)

# Filter out the excluded states from shipment data
excluded_states = ['Western Australia', 'Northern Territory', 'Tasmania', 'New Zealand']
filtered_shipment_data = shipment_data[~shipment_data['State'].isin(excluded_states)]

# Extract the necessary columns
landed_costs = landed_costs_data[['SKU', 'Landed Cost to AU']]
product_prices = actual_product_price_data[['Variant SKU', 'Variant Weight', 'Variant Price']]

# Merge the datasets on SKU
merged_data = pd.merge(landed_costs, product_prices, left_on='SKU', right_on='Variant SKU', how='inner')

# Calculate the adjusted final price using the weighted average shipping cost per kg
weighted_avg_shipping_cost_per_kg = 1.63  # The previously calculated average shipping cost per kg
merged_data['Adjusted Price'] = merged_data['Variant Price'] + (merged_data['Variant Weight'] * weighted_avg_shipping_cost_per_kg)

# Calculate initial and adjusted gross margins
merged_data['Initial Gross Margin'] = merged_data['Variant Price'] - merged_data['Landed Cost to AU']
merged_data['Adjusted Gross Margin'] = merged_data['Adjusted Price'] - merged_data['Landed Cost to AU']

# Process the shipment data to calculate total shipping costs per SKU
filtered_shipment_data['SKUs'] = filtered_shipment_data['Product'].str.split(',')
shipment_data_exploded = filtered_shipment_data.explode('SKUs')
shipment_data_exploded['SKUs'] = shipment_data_exploded['SKUs'].str.strip()

# Merge shipment data with SKU to get the relevant shipping costs
shipping_costs_merged = pd.merge(shipment_data_exploded, landed_costs, left_on='SKUs', right_on='SKU', how='inner')
shipping_costs_per_sku = shipping_costs_merged.groupby('SKU').agg(total_shipping_cost=('Total_Charge', 'sum')).reset_index()

# Merge shipping costs back into the main dataset
final_merged_data = pd.merge(merged_data, shipping_costs_per_sku, left_on='SKU', right_on='SKU', how='left')

# Display the first few rows of the final merged data
print(final_merged_data.head())

# Save the final merged data to a new Excel file
# final_merged_data.to_excel('Final_Merged_Data_with_Margins_and_Shipping_Costs.xlsx', index=False)


          SKU  Landed Cost to AU Variant SKU  Variant Weight  Variant Price  \
0  941-000152         386.630000  941-000152            7.15          594.0   
1  943-000125          41.420000  943-000125            0.36           39.0   
2  945-000024         197.230000  945-000024            3.52          287.0   
3  960-001090         132.530000  960-001090            0.34          157.0   
4    CSX3BLK4        1060.736343    CSX3BLK4            0.01         2449.0   

   Adjusted Price  Initial Gross Margin  Adjusted Gross Margin  \
0        605.6545            207.370000             219.024500   
1         39.5868             -2.420000              -1.833200   
2        292.7376             89.770000              95.507600   
3        157.5542             24.470000              25.024200   
4       2449.0163           1388.263657            1388.279957   

   total_shipping_cost  
0                  NaN  
1                  NaN  
2                  NaN  
3                  NaN  
4  

C:\Users\olcay\AppData\Local\Temp\ipykernel_17940\1381202329.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_shipment_data['SKUs'] = filtered_shipment_data['Product'].str.split(',')


In [18]:
import pandas as pd

# Load the final merged data
final_merged_data_path = 'Final_Merged_Data_with_Margins_and_Shipping_Costs.xlsx'
final_merged_data = pd.read_excel(final_merged_data_path)

# Calculate aggregated statistics
aggregated_stats = {
    'Total Products': final_merged_data['SKU'].nunique(),
    'Total Landed Cost': final_merged_data['Landed Cost to AU'].sum(),
    'Total Initial Gross Margin': final_merged_data['Initial Gross Margin'].sum(),
    'Total Adjusted Gross Margin': final_merged_data['Adjusted Gross Margin'].sum(),
    'Average Initial Gross Margin': final_merged_data['Initial Gross Margin'].mean(),
    'Average Adjusted Gross Margin': final_merged_data['Adjusted Gross Margin'].mean(),
    'Total Shipping Cost': final_merged_data['total_shipping_cost'].sum(),
    'Average Shipping Cost per Product': final_merged_data['total_shipping_cost'].mean()
}

# Display aggregated statistics
aggregated_stats_df = pd.DataFrame([aggregated_stats])
print(aggregated_stats_df)

   Total Products  Total Landed Cost  Total Initial Gross Margin  \
0             329       36692.358744                61379.641256   

   Total Adjusted Gross Margin  Average Initial Gross Margin  \
0                 62606.145351                    464.997282   

   Average Adjusted Gross Margin  Total Shipping Cost  \
0                      474.28898             158974.2   

   Average Shipping Cost per Product  
0                        1039.047059  


In [12]:
import pandas as pd

# Load the datasets
landed_costs_path = 'C:/Users/olcay/Desktop/TASKS/TrakRacer/AUS Shipping Comparison/Take a look later on/Landed Costs AUS.xlsx'
actual_product_price_path = 'C:/Users/olcay/Desktop/TASKS/TrakRacer/AUS Shipping Comparison/Take a look later on/Actual Product Price.xlsx'
shipment_data_path = 'C:/Users/olcay/Desktop/TASKS/TrakRacer/AUS Shipping Comparison/Updated_Data_with_Logistic_Regression_Predictions.csv'

# Read the data from the Excel files
landed_costs_data = pd.read_excel(landed_costs_path, sheet_name='All Product Cost')
actual_product_price_data = pd.read_excel(actual_product_price_path, sheet_name='Products')
shipment_data = pd.read_csv(shipment_data_path)

# Extract the necessary columns
landed_costs = landed_costs_data[['SKU', 'Landed Cost to AU']]
product_prices = actual_product_price_data[['Variant SKU', 'Variant Weight', 'Variant Price']]
shipment_data = shipment_data[['Order_Number', 'Product', 'Total_Charge', 'Date']]

# Replace '#REF!' and other non-numeric values with NaN
landed_costs['Landed Cost to AU'] = pd.to_numeric(landed_costs['Landed Cost to AU'], errors='coerce')

# Filter shipment data for June
shipment_data['Date'] = pd.to_datetime(shipment_data['Date'])
shipment_data_june = shipment_data[shipment_data['Date'].dt.month == 6]

# Process the shipment data to calculate total shipping costs per order
shipment_data_june['SKUs'] = shipment_data_june['Product'].str.split(',')
shipment_data_exploded = shipment_data_june.explode('SKUs')
shipment_data_exploded['SKUs'] = shipment_data_exploded['SKUs'].str.strip()

# Merge shipment data with product data
merged_data = pd.merge(shipment_data_exploded, product_prices, left_on='SKUs', right_on='Variant SKU', how='left')
merged_data = pd.merge(merged_data, landed_costs, left_on='SKUs', right_on='SKU', how='left')

# Calculate the adjusted final price using the weighted average shipping cost per kg
weighted_avg_shipping_cost_per_kg = 1.63
merged_data['Adjusted Price'] = merged_data['Variant Price'] + (merged_data['Variant Weight'] * weighted_avg_shipping_cost_per_kg)

# Calculate initial and adjusted gross margins
merged_data['Initial Gross Margin'] = merged_data['Variant Price'] - merged_data['Landed Cost to AU']
merged_data['Adjusted Gross Margin'] = merged_data['Adjusted Price'] - merged_data['Landed Cost to AU']

# Fill N/A values for margins
merged_data['Initial Gross Margin'] = merged_data['Initial Gross Margin'].fillna(0)
merged_data['Adjusted Gross Margin'] = merged_data['Adjusted Gross Margin'].fillna(0)

# Calculate the adjusted shipping cost for each SKU
merged_data['Adjusted Shipping Cost'] = merged_data['Variant Weight'] * weighted_avg_shipping_cost_per_kg

# Summarize total gross margins and shipping costs per order
order_summary = merged_data.groupby('Order_Number').agg(
    total_initial_gross_margin=('Initial Gross Margin', 'sum'),
    total_adjusted_gross_margin=('Adjusted Gross Margin', 'sum'),
    total_actual_shipping_cost=('Total_Charge', 'sum'),
    total_adjusted_shipping_cost=('Adjusted Shipping Cost', 'sum')
).reset_index()

# Calculate average values
average_initial_gross_margin = order_summary['total_initial_gross_margin'].mean()
average_adjusted_gross_margin = order_summary['total_adjusted_gross_margin'].mean()
average_gross_margin_increase = average_adjusted_gross_margin - average_initial_gross_margin
average_actual_shipping_cost_per_order = order_summary['total_actual_shipping_cost'].mean()
average_adjusted_shipping_cost_per_order = order_summary['total_adjusted_shipping_cost'].mean()

# Display results
print(f"Average Initial Gross Margin: {average_initial_gross_margin:.2f}")
print(f"Average Adjusted Gross Margin: {average_adjusted_gross_margin:.2f}")
print(f"Average Increase in Gross Margin: {average_gross_margin_increase:.2f}")
print(f"Average Actual Shipping Cost per Order: {average_actual_shipping_cost_per_order:.2f}")
print(f"Average Adjusted Shipping Cost per Order: {average_adjusted_shipping_cost_per_order:.2f}")

# Save the detailed comparison to a new Excel file
# order_summary.to_excel('Order_Summary_with_Gross_Margins_and_Shipping_Costs.xlsx', index=False)


Average Initial Gross Margin: 96.19
Average Adjusted Gross Margin: 97.42
Average Increase in Gross Margin: 1.22
Average Actual Shipping Cost per Order: 286.36
Average Adjusted Shipping Cost per Order: 23.89


C:\Users\olcay\AppData\Local\Temp\ipykernel_17940\2662319410.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  landed_costs['Landed Cost to AU'] = pd.to_numeric(landed_costs['Landed Cost to AU'], errors='coerce')


# Final

In [ ]:
import pandas as pd

# Load the data from the provided Excel file
file_path = '/mnt/data/Order_AUS.xlsx'
data = pd.read_excel(file_path)

# Define the excluded states
excluded_states = ["Western Australia", "Northern Territory", "Tasmania", "New Zealand"]

# Filter out orders with 0.00 shipping cost
filtered_data = data[data['Shipping'] > 0]
# Filter out orders with 0.00 shipping cost
filtered_data = data[data['Shipping'] > 0]

# Calculate the adjusted subtotal after applying the discount
filtered_data['adjusted_subtotal'] = filtered_data['Lineitem price'] - filtered_data['Discount Amount']

# Calculate the required increase percentage for each order to cover shipping costs
filtered_data['required_increase_percentage'] = (filtered_data['Shipping'] / filtered_data['adjusted_subtotal']) * 100

# Compute the median of the required increase percentages
median_required_increase_percentage = filtered_data['required_increase_percentage'].median()

# Compute the weighted average of the required increase percentages
weighted_avg_required_increase_percentage = (filtered_data['required_increase_percentage'] * filtered_data['adjusted_subtotal']).sum() / filtered_data['adjusted_subtotal'].sum()

# Calculate the adjusted increase percentage
adjusted_increase_percentage = (median_required_increase_percentage + weighted_avg_required_increase_percentage) / 2

# Display the calculated percentages
print("Median Required Increase Percentage:", median_required_increase_percentage)
print("Weighted Average Required Increase Percentage:", weighted_avg_required_increase_percentage)
print("Adjusted Increase Percentage:", adjusted_increase_percentage)

Working from the final Data